## 예제 1. 기본 프롬프트 체인 다시 만들기

In [1]:
!pip install -q langchain-core langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00


In [2]:
import os

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
# 1. 모델 정의
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-5-mini",
)

In [5]:
# 2. 각 체인 링크에 대한 프롬프트 템플릿 정의
prompt_summarize = ChatPromptTemplate.from_template(
    "다음 기술 문서를 세 문장으로 간결하게 요약해주세요:\n\n[원본 문서]\n{document}"
)

prompt_create_script = ChatPromptTemplate.from_template(
    "다음 요약문을 바탕으로, 비즈니스팀을 위한 1분 발표 스크립트를 작성해주세요:\n\n[요약문]\n{summary}"
)

prompt_extract_keywords = ChatPromptTemplate.from_template(
    "다음 발표 스크립트에서 가장 중요한 핵심 키워드 5개를 쉼표(,)로 구분하여 나열해주세요:\n\n[발표 스크립트 전문]\n{script}"
)

In [6]:
# 3. 출력 파서 정의
# LLM의 출력(ChatMessage)에서 내용(content)만 추출하여 문자열로 변환한다.
output_parser = StrOutputParser()

In [7]:
# 4. 파이프( | )를 사용하여 전체 체인 구성
# {"summary": summarize_chain} 구문은 summarize_chain의 결과를 다음 체인의 'summary' 입력으로 전달하라는 의미이다.
summarize_chain = prompt_summarize | llm | output_parser
script_chain = {"summary": summarize_chain} | prompt_create_script | llm | output_parser
keyword_chain = {"script": script_chain} | prompt_extract_keywords | llm | output_parser

In [8]:
# --- 체인 실행 ---
original_document = """
검색 증강 생성(Retrieval-Augmented Generation, RAG)은 대규모 언어 모델(LLM)의 할루시네이션(환각) 현상을 줄이고, 답변의 근거를 외부의 최신 정보에 두기 위한 강력한 아키텍처이다. RAG는 사용자의 질문이 들어오면 LLM이 바로 답변을 생성하는 것이 아니라, 먼저 외부 지식 소스(예: 벡터 데이터베이스)에서 질문과 관련성 높은 문서를 실시간으로 검색(Retrieve)한다. 그런 다음, 검색된 최신 문서를 원래의 질문과 함께 프롬프트에 포함하여 LLM에게 전달함으로써, 모델이 외부의 사실적 근거를 바탕으로 답변을 생성(Generate)하도록 유도한다. 이 방식은 LLM의 내부 학습 데이터가 특정 시점에 고정되어 발생하는 정보의 시의성 부족과 부정확성 문제를 효과적으로 해결하며, 답변의 출처를 명시할 수 있어 신뢰도를 높인다.
"""

# 최종 체인(keyword_chain)에 초기 입력을 전달하여 실행한다.
final_keywords = keyword_chain.invoke({"document": original_document})

print("========================================")
print("LangChain으로 구성된 체인 실행 완료")
print(f"최종 추출 키워드: {final_keywords}")
print("========================================")

LangChain으로 구성된 체인 실행 완료
최종 추출 키워드: 검색 증강 생성(RAG), 환각, 벡터 DB, 근거 기반 답변, 출처 명시


## 예제 2: 간단한 RAG(검색 증강 생성) 체인 구축하기

In [9]:
!pip install -q beautifulsoup4 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.1 MB/s eta 0:00:00


In [10]:
import os

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from google.colab import userdata

In [11]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [12]:
# 1. 데이터 로드 (Load): 웹 페이지의 내용을 불러온다.
loader = WebBaseLoader("https://www.anthropic.com/engineering/building-effective-agents")
docs = loader.load()

In [13]:
# 2. 텍스트 분할 (Split): 불러온 문서를 의미 있는 작은 조각(Chunk)으로 나눈다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [14]:
# 3. 임베딩 및 벡터 저장소 생성 (Embed & Store): 각 텍스트 조각을 벡터로 변환하고, 검색 가능한 벡터 저장소에 저장한다.
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)

In [15]:
# 4. 검색기(Retriever) 생성: 벡터 저장소에서 관련성 높은 문서를 검색하는 객체를 만든다.
retriever = vectorstore.as_retriever()

In [16]:
# 5. RAG 프롬프트 템플릿 정의
# 검색된 문서(context)와 사용자 질문(question)을 모두 입력으로 받는다.
prompt_rag = ChatPromptTemplate.from_template(
    """당신은 주어진 문맥을 바탕으로 질문에 답변하는 AI 어시스턴트입니다.
문맥의 내용만을 사용하여 답변해주세요.


문맥: {context}

질문: {question}

답변:"""
)

In [17]:
# 6. RAG 체인 구성
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-5",
)

In [18]:
# 검색된 문서들을 하나의 문자열로 합치는 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [19]:
# LCEL을 사용하여 RAG 체인을 선언적으로 구성한다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [20]:
# --- RAG 체인 실행 ---
question = "에이전트를 효과적으로 구성하기 위해 검토할 사항이 뭐가 있을까요?"
final_answer = rag_chain.invoke(question)

print("========================================")
print(f"질문: {question}")
print(f"답변: {final_answer}")
print("========================================")

질문: 에이전트를 효과적으로 구성하기 위해 검토할 사항이 뭐가 있을까요?
답변: 다음 사항을 점검하면 에이전트를 효과적으로 구성할 수 있습니다.

- 문제 적합성: 대화와 행동이 모두 필요한 업무, 성공 기준이 명확하고 피드백 루프와 인간 감독이 가능한 업무를 우선 선정.
- 모델 역량 전제: 복잡한 입력 이해, 추론·계획, 도구의 신뢰성 있는 사용, 오류로부터의 회복.
- 단순성 우선: 간단한 프롬프트로 시작해 평가로 성능을 측정하고, 단순한 해법이 부족할 때만 다단계 에이전틱 설계를 추가.
- 투명성: 에이전트의 계획 단계를 명시적으로 드러내기.
- ACI(Agent-Computer Interface) 설계: 도구의 구조를 정확히 정의하고, 문서화와 테스트로 신뢰성을 확보.
- 도구 프롬프트 엔지니어링: 외부 서비스/API 상호작용을 위해 도구의 정확한 스키마를 제공.
- 실행 중 검증: 각 단계마다 도구 호출 결과나 코드 실행 등 환경으로부터 “그라운드 트루스”를 받아 진행 상황을 평가.
- 인간 참여: 체크포인트나 장애 상황에서 사용자 피드백을 받는 절차 마련.
- 중단 조건: 최대 반복 횟수 등 명시적 스토핑 조건 설정.
- 운영 방식: 사용자의 명령/대화로 과업을 명확히 한 뒤 독립적으로 계획·실행하되 필요 시 사용자에게 재질의.
- 성능 관리: 측정과 반복을 통해 개선하고, 복잡성 추가는 실제 성과 향상 시에만.
- 프로덕션 고려: 프레임워크로 빠르게 시작하되, 필요하면 추상화 레이어를 줄이고 기본 컴포넌트로 전환.
- 활용 예 기준: 
  - 고객지원: 챗봇 인터페이스에 도구를 결합하고 의미 있는 인간 감독을 통합.
  - 코딩 에이전트: 자동 테스트로 검증 가능, 테스트 결과로 반복 개선, 객관적 품질 측정, 광범위한 요구 정렬을 위한 인간 검토 유지.
